In [ ]:
# Document loading, retrieval methods and text splitting
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

# Web Loader
%pip install -qU beautifulsoup4

In [ ]:
import os
import getpass

# Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# OR (load from .env file)

# from dotenv import load_dotenv
# load_dotenv("./.env")

In [4]:
MODEL = "llama3.2"

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.document_loaders import PyPDFLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [6]:
from langchain_community.document_loaders import WebBaseLoader


links = ["https://python.langchain.com/docs/how_to/structured_output/",
         "https://python.langchain.com/docs/how_to/tool_calling/",
         "https://python.langchain.com/docs/how_to/few_shot_examples/",
         "https://python.langchain.com/docs/how_to/prompts_composition/",
         "https://python.langchain.com/docs/how_to/functions/",
         "https://python.langchain.com/docs/how_to/parallel/",
         "https://python.langchain.com/docs/how_to/sequence/",
         "https://python.langchain.com/docs/concepts/#langchain-expression-language-lcel",
         "https://python.langchain.com/docs/how_to/installation/",
         "https://python.langchain.com/docs/how_to/document_loader_markdown/",
         "https://python.langchain.com/docs/how_to/document_loader_json/",      
         "https://python.langchain.com/docs/how_to/document_loader_pdf/",
         "https://python.langchain.com/docs/how_to/document_loader_web/",
         "https://python.langchain.com/docs/how_to/document_loader_csv/",
         "https://python.langchain.com/docs/how_to/document_loader_directory/", 
         "https://python.langchain.com/docs/how_to/document_loader_html/",
         "https://python.langchain.com/docs/tutorials/rag/"
         ]

loader = WebBaseLoader(links)

In [7]:
langchain_docs = loader.load_and_split() # SPLIT
langchain_docs

[Document(metadata={'source': 'https://python.langchain.com/docs/how_to/structured_output/', 'title': 'How to return structured data from a model | 🦜️🔗 LangChain', 'description': 'This guide assumes familiarity with the following concepts:', 'language': 'en'}, page_content='How to return structured data from a model | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/how_to/structured_output/', 'title': 'How to return structured data from a model | 🦜️🔗 LangChain', 'description': 'This guide assumes familiarity with the following concepts:', 'language': 'en'}, page_content='Skip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generatio

In [8]:
from langchain_core.documents import Document
from collections import defaultdict

# Group documents by source
grouped_docs = defaultdict(list)
for doc in langchain_docs:
    source = doc.metadata.get('source', '')
    grouped_docs[source].append(doc)

# Combine documents with the same source
combined_docs = []
for source, docs in grouped_docs.items():
    combined_content = "\n".join(doc.page_content for doc in docs)
    combined_metadata = docs[0].metadata.copy()  # Use metadata from the first document
    combined_metadata['num_chunks'] = len(docs)  # Add number of original chunks
    combined_docs.append(Document(page_content=combined_content, metadata=combined_metadata))

# Replace langchain_docs with the combined documents
langchain_docs = combined_docs

In [9]:
len(langchain_docs)

17

In [10]:
doc_obj = langchain_docs[0]
doc_obj

Document(metadata={'source': 'https://python.langchain.com/docs/how_to/structured_output/', 'title': 'How to return structured data from a model | 🦜️🔗 LangChain', 'description': 'This guide assumes familiarity with the following concepts:', 'language': 'en', 'num_chunks': 12}, page_content='How to return structured data from a model | 🦜️🔗 LangChain\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools

In [11]:
type(doc_obj)

langchain_core.documents.base.Document

In [12]:
doc_obj.page_content

'How to return structured data from a model | 🦜️🔗 LangChain\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to map values to a graph databaseHow to add a semantic layer over graph databaseHow to invoke runnables in parallelHow to stream chat model responsesHow to add default invocation a

In [13]:
len(langchain_docs)

17

In [14]:
from IPython.display import display, Markdown

Markdown(doc_obj.page_content)

How to return structured data from a model | 🦜️🔗 LangChain
Skip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to map values to a graph databaseHow to add a semantic layer over graph databaseHow to invoke runnables in parallelHow to stream chat model responsesHow to add default invocation args to a RunnableHow to add retrieval to chatbotsHow to use few shot examples in chat modelsHow to do tool/function callingHow to best prompt for Graph-RAGHow to install LangChain packagesHow to add examples to the prompt for query analysisHow to use few shot examplesHow to run custom functionsHow to use output parsers to parse an LLM response into structured formatHow to handle cases where no queries are generatedHow to route between sub-chainsHow to return structured data from a modelHow to summarize text through parallelizationHow to summarize text through iterative refinementHow to summarize text in a single LLM callHow to use toolkitsHow to add ad-hoc tool calling capability to LLMs and Chat ModelsBuild an Agent with AgentExecutor (Legacy)How to construct knowledge graphsHow to partially format prompt templatesHow to handle multiple queries when doing query analysisHow to use built-in tools and toolkitsHow to pass through arguments from one step to the nextHow to compose prompts togetherHow to handle multiple retrievers when doing query analysisHow to add values to a chain's stateHow to construct filters for query analysisHow to configure runtime chain internalsHow deal with high cardinality categoricals when doing query analysisCustom Document LoaderHow to split by HTML headerHow to split by HTML sectionsHow to use the MultiQueryRetrieverHow to add scores to retriever resultsCachingHow to use callbacks in async environmentsHow to attach callbacks to a runnableHow to propagate callbacks  constructorHow to dispatch custom callback eventsHow to pass callbacks in at runtimeHow to split by characterHow to cache chat model responsesHow to handle rate limitsHow to init any model in one lineHow to track token usage in ChatModelsHow to add tools to chatbotsHow to split codeHow to do retrieval with contextual compressionHow to convert Runnables as ToolsHow to create custom callback handlersHow to create a custom chat model classHow to create a custom LLM classCustom RetrieverHow to create toolsHow to debug your LLM appsHow to load CSVsHow to load documents from a directoryHow to load HTMLHow to load JSONHow to load MarkdownHow to load Microsoft Office filesHow to load PDFsHow to load web pagesHow to create a dynamic (self-constructing) chainText embedding modelsHow to combine results from multiple retrieversHow to select examples from a LangSmith datasetHow to select examples by lengthHow to select examples by maximal marginal relevance (MMR)How to select examples by n-gram overlapHow to select examples by similarityHow to use reference examples when doing extractionHow to handle long text when doing extractionHow to use prompting alone (no tool calling) to do extractionHow to add fallbacks to a runnableHow to filter messagesHybrid SearchHow to use the LangChain indexing APIHow to inspect runnablesLangChain Expression Language CheatsheetHow to cache LLM responsesHow to track token usage for LLMsRun models locallyHow to get log probabilitiesHow to reorder retrieved results to mitigate the "lost in the middle"
Language CheatsheetHow to cache LLM responsesHow to track token usage for LLMsRun models locallyHow to get log probabilitiesHow to reorder retrieved results to mitigate the "lost in the middle" effectHow to split Markdown by HeadersHow to merge consecutive messages of the same typeHow to add message historyHow to migrate from legacy LangChain agents to LangGraphHow to retrieve using multiple vectors per documentHow to pass multimodal data directly to modelsHow to use multimodal promptsHow to create a custom Output ParserHow to use the output-fixing parserHow to parse JSON outputHow to retry when a parsing error occursHow to parse XML outputHow to parse YAML outputHow to use the Parent Document RetrieverHow to use LangChain with different Pydantic versionsHow to add chat historyHow to get a RAG application to add citationsHow to do per-user retrievalHow to get your RAG application to return sourcesHow to stream results from your RAG applicationHow to split JSON dataHow to recursively split text by charactersResponse metadataHow to pass runtime secrets to runnablesHow to do "self-querying" retrievalHow to split text based on semantic similarityHow to chain runnablesHow to save and load LangChain objectsHow to split text by tokensHow to do question answering over CSVsHow to deal with large databases when doing SQL question-answeringHow to better prompt when doing SQL question-answeringHow to do query validation as part of SQL question-answeringHow to stream runnablesHow to stream responses from an LLMHow to use a time-weighted vector store retrieverHow to return artifacts from a toolHow to use chat models to call toolsHow to disable parallel tool callingHow to force models to call a toolHow to access the RunnableConfig from a toolHow to pass tool outputs to chat modelsHow to pass run time values to toolsHow to stream events from a toolHow to stream tool callsHow to convert tools to OpenAI FunctionsHow to handle tool errorsHow to use few-shot prompting with tool callingHow to add a human-in-the-loop for toolsHow to bind model-specific toolsHow to trim messagesHow to create and query vector storesConceptual guideEcosystem🦜🛠️ LangSmith🦜🕸️ LangGraphVersionsv0.3v0.2Pydantic compatibilityMigrating from v0.0 chainsHow to migrate from v0.0 chainsMigrating from ConstitutionalChainMigrating from ConversationalChainMigrating from ConversationalRetrievalChainMigrating from LLMChainMigrating from LLMMathChainMigrating from LLMRouterChainMigrating from MapReduceDocumentsChainMigrating from MapRerankDocumentsChainMigrating from MultiPromptChainMigrating from RefineDocumentsChainMigrating from RetrievalQAMigrating from StuffDocumentsChainUpgrading to LangGraph memoryHow to migrate to LangGraph memoryHow to use BaseChatMessageHistory with LangGraphMigrating off ConversationBufferMemory or ConversationStringBufferMemoryMigrating off ConversationBufferWindowMemory or ConversationTokenBufferMemoryMigrating off ConversationSummaryMemory or ConversationSummaryBufferMemoryA Long-Term Memory AgentRelease policySecurityHow-to guidesHow to return structured data from a modelOn this pageHow to return structured data from a model
PrerequisitesThis guide assumes familiarity with the following concepts:
Chat models
Function/tool calling
It is often useful to have a model return output that matches a specific schema. One common use-case is extracting data from text to insert into a database or use with some other downstream system. This guide covers a few strategies for getting structured outputs from a model.
The .with_structured_output() method​

Supported modelsYou can find a list of models that support this method here.
This is the easiest and most reliable way to get structured outputs. with_structured_output() is implemented for models that provide native APIs for structuring outputs, like tool/function calling or JSON mode, and makes use of these capabilities under the hood.
This method takes a schema as input which specifies the names, types, and descriptions of the desired output attributes. The method returns a model-like Runnable, except that instead of outputting strings or Messages it outputs objects corresponding to the given schema. The schema can be specified as a TypedDict class, JSON Schema or a Pydantic class. If TypedDict or JSON Schema are used then a dictionary will be returned by the Runnable, and if a Pydantic class is used then a Pydantic object will be returned.
As an example, let's get a model to generate a joke and separate the setup from the punchline:
OpenAIAnthropicAzureGoogleCohereNVIDIAFireworksAIGroqMistralAITogetherAIpip install -qU langchain-openaiimport getpassimport osos.environ["OPENAI_API_KEY"] = getpass.getpass()from langchain_openai import ChatOpenAIllm = ChatOpenAI(model="gpt-4o-mini")pip install -qU langchain-anthropicimport getpassimport osos.environ["ANTHROPIC_API_KEY"] = getpass.getpass()from langchain_anthropic import ChatAnthropicllm = ChatAnthropic(model="claude-3-5-sonnet-20240620")pip install -qU langchain-openaiimport getpassimport osos.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass()from langchain_openai import AzureChatOpenAIllm = AzureChatOpenAI(    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],)pip install -qU langchain-google-vertexaiimport getpassimport osos.environ["GOOGLE_API_KEY"] = getpass.getpass()from langchain_google_vertexai import ChatVertexAIllm = ChatVertexAI(model="gemini-1.5-flash")pip install -qU langchain-cohereimport getpassimport osos.environ["COHERE_API_KEY"] = getpass.getpass()from langchain_cohere import ChatCoherellm = ChatCohere(model="command-r-plus")pip install -qU langchain-nvidia-ai-endpointsimport getpassimport osos.environ["NVIDIA_API_KEY"] = getpass.getpass()from langchain import ChatNVIDIAllm = ChatNVIDIA(model="meta/llama3-70b-instruct")pip install -qU langchain-fireworksimport getpassimport osos.environ["FIREWORKS_API_KEY"] = getpass.getpass()from langchain_fireworks import ChatFireworksllm = ChatFireworks(model="accounts/fireworks/models/llama-v3p1-70b-instruct")pip install -qU langchain-groqimport getpassimport osos.environ["GROQ_API_KEY"] = getpass.getpass()from langchain_groq import ChatGroqllm = ChatGroq(model="llama3-8b-8192")pip install -qU langchain-mistralaiimport getpassimport osos.environ["MISTRAL_API_KEY"] = getpass.getpass()from langchain_mistralai import ChatMistralAIllm = ChatMistralAI(model="mistral-large-latest")pip install -qU langchain-openaiimport getpassimport osos.environ["TOGETHER_API_KEY"] = getpass.getpass()from langchain_openai import ChatOpenAIllm = ChatOpenAI(    base_url="https://api.together.xyz/v1",    api_key=os.environ["TOGETHER_API_KEY"],    model="mistralai/Mixtral-8x7B-Instruct-v0.1",)
Pydantic class​
If we want the model to return a Pydantic object, we just need to pass in the desired Pydantic class. The key advantage of using Pydantic is that the model-generated output will be validated. Pydantic will raise an error if any required fields are missing or if any fields are of the wrong type.
from typing import Optionalfrom pydantic import BaseModel, Field# Pydanticclass Joke(BaseModel):    """Joke to tell user."""    setup: str = Field(description="The setup of the joke")    punchline: str = Field(description="The punchline to the joke")    rating: Optional[int] = Field(        default=None, description="How funny the joke is, from 1 to 10"    )structured_llm = llm.with_structured_output(Joke)structured_llm.invoke("Tell me a joke about cats")
Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7)
tipBeyond just the structure of the Pydantic class, the name of the Pydantic class, the docstring, and the names and provided descriptions of parameters are very important. Most of the time with_structured_output is using a model's function/tool calling API, and you can effectively think of all of this information as being added to the model prompt.
TypedDict or JSON Schema​
TypedDict or JSON Schema​
If you don't want to use Pydantic, explicitly don't want validation of the arguments, or want to be able to stream the model outputs, you can define your schema using a TypedDict class. We can optionally use a special Annotated syntax supported by LangChain that allows you to specify the default value and description of a field. Note, the default value is not filled in automatically if the model doesn't generate it, it is only used in defining the schema that is passed to the model.
Requirements
Core: langchain-core>=0.2.26
Typing extensions: It is highly recommended to import Annotated and TypedDict from typing_extensions instead of typing to ensure consistent behavior across Python versions.
from typing_extensions import Annotated, TypedDict# TypedDictclass Joke(TypedDict):    """Joke to tell user."""    setup: Annotated[str, ..., "The setup of the joke"]    # Alternatively, we could have specified setup as:    # setup: str                    # no default, no description    # setup: Annotated[str, ...]    # no default, no description    # setup: Annotated[str, "foo"]  # default, no description    punchline: Annotated[str, ..., "The punchline of the joke"]    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]structured_llm = llm.with_structured_output(Joke)structured_llm.invoke("Tell me a joke about cats")
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!', 'rating': 7}
Equivalently, we can pass in a JSON Schema dict. This requires no imports or classes and makes it very clear exactly how each parameter is documented, at the cost of being a bit more verbose.
json_schema = {    "title": "joke",    "description": "Joke to tell user.",    "type": "object",    "properties": {        "setup": {            "type": "string",            "description": "The setup of the joke",        },        "punchline": {            "type": "string",            "description": "The punchline to the joke",        },        "rating": {            "type": "integer",            "description": "How funny the joke is, from 1 to 10",            "default": None,        },    },    "required": ["setup", "punchline"],}structured_llm = llm.with_structured_output(json_schema)structured_llm.invoke("Tell me a joke about cats")
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!', 'rating': 7}
Choosing between multiple schemas​
The simplest way to let the model choose from multiple schemas is to create a parent schema that has a Union-typed attribute:
from typing import Union# Pydanticclass Joke(BaseModel):    """Joke to tell user."""    setup: str = Field(description="The setup of the joke")    punchline: str = Field(description="The punchline to the joke")    rating: Optional[int] = Field(        default=None, description="How funny the joke is, from 1 to 10"    )class ConversationalResponse(BaseModel):    """Respond in a conversational manner. Be kind and helpful."""    response: str = Field(description="A conversational response to the user's query")class FinalResponse(BaseModel):    final_output: Union[Joke, ConversationalResponse]structured_llm = llm.with_structured_output(FinalResponse)structured_llm.invoke("Tell me a joke about cats")
FinalResponse(final_output=Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7))
structured_llm.invoke("How are you today?")
FinalResponse(final_output=ConversationalResponse(response="I'm just a bunch of code, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?"))
Alternatively, you can use tool calling directly to allow the model to choose between options, if your chosen model supports it. This involves a bit more parsing and setup but in some instances leads to better performance because you don't have to use nested schemas. See this how-to guide for more details.
Streaming​
We can stream outputs from our structured model when the output type is a dict (i.e., when the schema is specified as a TypedDict class or  JSON Schema dict).
infoNote that what's yielded is already aggregated chunks, not deltas.
from typing_extensions import Annotated, TypedDict# TypedDictclass Joke(TypedDict):    """Joke to tell user."""    setup: Annotated[str, ..., "The setup of the joke"]    punchline: Annotated[str, ..., "The punchline of the joke"]    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]structured_llm = llm.with_structured_output(Joke)for chunk in structured_llm.stream("Tell me a joke about cats"):    print(chunk)
{}{'setup': ''}{'setup': 'Why'}{'setup': 'Why was'}{'setup': 'Why was the'}{'setup': 'Why was the cat'}{'setup': 'Why was the cat sitting'}{'setup': 'Why was the cat sitting on'}{'setup': 'Why was the cat sitting on the'}{'setup': 'Why was the cat sitting on the computer'}{'setup': 'Why was the cat sitting on the computer?'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': ''}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!'}{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!', 'rating': 7}
Few-shot prompting​
For more complex schemas it's very useful to add few-shot examples to the prompt. This can be done in a few ways.
The simplest and most universal way is to add examples to a system message in the prompt:
from langchain_core.prompts import ChatPromptTemplatesystem = """You are a hilarious comedian. Your specialty is knock-knock jokes. \Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").Here are some examples of jokes:example_user: Tell me a joke about planesexample_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}example_user: Tell me another joke about planesexample_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}example_user: Now about caterpillarsexample_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])few_shot_structured_llm = prompt | structured_llmfew_shot_structured_llm.invoke("what's something funny about woodpeckers")API Reference:ChatPromptTemplate
{'setup': 'Woodpecker', 'punchline': "Woodpecker who? Woodpecker who can't find a tree is just a bird with a headache!", 'rating': 7}
When the underlying method for structuring outputs is tool calling, we can pass in our examples as explicit tool calls. You can check if the model you're using makes use of tool calling in its API reference.
from langchain_core.messages import AIMessage, HumanMessage, ToolMessageexamples = [    HumanMessage("Tell me a joke about planes", name="example_user"),    AIMessage(        "",        name="example_assistant",        tool_calls=[            {                "name": "joke",                "args": {                    "setup": "Why don't planes ever get tired?",                    "punchline": "Because they have rest wings!",                    "rating": 2,                },                "id": "1",            }        ],    ),    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.    ToolMessage("", tool_call_id="1"),    # Some models also expect an AIMessage to follow any ToolMessages,    # so you may need to add an AIMessage here.    HumanMessage("Tell me another joke about planes", name="example_user"),    AIMessage(        "",        name="example_assistant",        tool_calls=[            {                "name": "joke",                "args": {                    "setup": "Cargo",                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",                    "rating": 10,                },                "id": "2",            }        ],    ),    ToolMessage("", tool_call_id="2"),    HumanMessage("Now about caterpillars", name="example_user"),    AIMessage(        "",        tool_calls=[            {                "name": "joke",                "args": {                    "setup": "Caterpillar",                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",                    "rating": 5,                },                "id": "3",            }        ],    ),    ToolMessage("", tool_call_id="3"),]system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \Return a joke which has the setup (the response to "Who's there?") \and the final punchline (the response to "<setup> who?")."""prompt = ChatPromptTemplate.from_messages(    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")])few_shot_structured_llm = prompt | structured_llmfew_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})API Reference:AIMessage | HumanMessage | ToolMessage
{'setup': 'Crocodile', 'punchline': 'Crocodile be seeing you later, alligator!', 'rating': 7}
For more on few shot prompting when using tool calling, see here.
(Advanced) Specifying the method for structuring outputs​
For models that support more than one means of structuring outputs (i.e., they support both tool calling and JSON mode), you can specify which method to use with the method= argument.
JSON modeIf using JSON mode you'll have to still specify the desired schema in the model prompt. The schema you pass to with_structured_output will only be used for parsing the model outputs, it will not be passed to the model the way it is with tool calling.To see if the model you're using supports JSON mode, check its entry in the API reference.
structured_llm = llm.with_structured_output(None, method="json_mode")structured_llm.invoke(    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys")
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!'}
(Advanced) Raw outputs​
LLMs aren't perfect at generating structured output, especially as schemas become complex. You can avoid raising exceptions and handle the raw output yourself by passing include_raw=True. This changes the output format to contain the raw message output, the parsed value (if successful), and any resulting errors:
structured_llm = llm.with_structured_output(Joke, include_raw=True)structured_llm.invoke("Tell me a joke about cats")
structured_llm = llm.with_structured_output(Joke, include_raw=True)structured_llm.invoke("Tell me a joke about cats")
{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f25ZRmh8u5vHlOWfTUw8sJFZ', 'function': {'arguments': '{"setup":"Why was the cat sitting on the computer?","punchline":"Because it wanted to keep an eye on the mouse!","rating":7}', 'name': 'Joke'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 93, 'total_tokens': 126}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-d880d7e2-df08-4e9e-ad92-dfc29f2fd52f-0', tool_calls=[{'name': 'Joke', 'args': {'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!', 'rating': 7}, 'id': 'call_f25ZRmh8u5vHlOWfTUw8sJFZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 93, 'output_tokens': 33, 'total_tokens': 126}), 'parsed': {'setup': 'Why was the cat sitting on the computer?',  'punchline': 'Because it wanted to keep an eye on the mouse!',  'rating': 7}, 'parsing_error': None}
Prompting and parsing model outputs directly​
Not all models support .with_structured_output(), since not all models have tool calling or JSON mode support. For such models you'll need to directly prompt the model to use a specific format, and use an output parser to extract the structured response from the raw model output.
Using PydanticOutputParser​
The following example uses the built-in PydanticOutputParser to parse the output of a chat model prompted to match the given Pydantic schema. Note that we are adding format_instructions directly to the prompt from a method on the parser:
from typing import Listfrom langchain_core.output_parsers import PydanticOutputParserfrom langchain_core.prompts import ChatPromptTemplatefrom pydantic import BaseModel, Fieldclass Person(BaseModel):    """Information about a person."""    name: str = Field(..., description="The name of the person")    height_in_meters: float = Field(        ..., description="The height of the person expressed in meters."    )class People(BaseModel):    """Identifying information about all people in a text."""    people: List[Person]# Set up a parserparser = PydanticOutputParser(pydantic_object=People)# Promptprompt = ChatPromptTemplate.from_messages(    [        (            "system",            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",        ),        ("human", "{query}"),    ]).partial(format_instructions=parser.get_format_instructions())API Reference:PydanticOutputParser | ChatPromptTemplate
Let’s take a look at what information is sent to the model:
query = "Anna is 23 years old and she is 6 feet tall"print(prompt.invoke(query).to_string())
System: Answer the user query. Wrap the output in `json` tagsThe output should be formatted as a JSON instance that conforms to the JSON schema below.As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.Here is the output schema:\`\`\`{"description": "Identifying information about all people in a text.", "properties": {"people": {"title": "People", "type": "array", "items": {"$ref": "#/definitions/Person"}}}, "required": ["people"], "definitions": {"Person": {"title": "Person", "description": "Information about a person.", "type": "object", "properties": {"name": {"title": "Name", "description": "The name of the person", "type": "string"}, "height_in_meters": {"title": "Height In Meters", "description": "The height of the person expressed in meters.", "type": "number"}}, "required": ["name", "height_in_meters"]}}}\`\`\`Human: Anna is 23 years old and she is 6 feet tall
And now let's invoke it:
chain = prompt | llm | parserchain.invoke({"query": query})
People(people=[Person(name='Anna', height_in_meters=1.8288)])
For a deeper dive into using output parsers with prompting techniques for structured output, see this guide.
Custom Parsing​
You can also create a custom prompt and parser with LangChain Expression Language (LCEL), using a plain function to parse the output from the model:
import jsonimport refrom typing import Listfrom langchain_core.messages import AIMessagefrom langchain_core.prompts import ChatPromptTemplatefrom pydantic import BaseModel, Fieldclass Person(BaseModel):    """Information about a person."""    name: str = Field(..., description="The name of the person")    height_in_meters: float = Field(        ..., description="The height of the person expressed in meters."    )class People(BaseModel):    """Identifying information about all people in a text."""    people: List[Person]# Promptprompt = ChatPromptTemplate.from_messages(    [        (            "system",            "Answer the user query. Output your answer as JSON that  "            "matches the given schema: \`\`\`json\n{schema}\n\`\`\`. "            "Make sure to wrap the answer in \`\`\`json and \`\`\` tags",        ),        ("human", "{query}"),    ]).partial(schema=People.schema())# Custom parserdef extract_json(message: AIMessage) -> List[dict]:    """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.    Parameters:        text (str): The text containing the JSON content.    Returns:        list: A list of extracted JSON strings.    """    text = message.content    # Define the regular expression pattern to match JSON blocks    pattern = r"\`\`\`json(.*?)\`\`\`"    # Find all non-overlapping matches of the pattern in the string    matches = re.findall(pattern, text, re.DOTALL)    # Return the list of matched JSON strings, stripping any leading or trailing whitespace    try:        return [json.loads(match.strip()) for match in matches]    except Exception:        raise ValueError(f"Failed to parse: {message}")API Reference:AIMessage | ChatPromptTemplate
Here is the prompt sent to the model:
query = "Anna is 23 years old and she is 6 feet tall"print(prompt.format_prompt(query=query).to_string())
System: Answer the user query. Output your answer as JSON that  matches the given schema: \`\`\`json{'title': 'People', 'description': 'Identifying information about all people in a text.', 'type': 'object', 'properties': {'people': {'title': 'People', 'type': 'array', 'items': {'$ref': '#/definitions/Person'}}}, 'required': ['people'], 'definitions': {'Person': {'title': 'Person', 'description': 'Information about a person.', 'type': 'object', 'properties': {'name': {'title': 'Name', 'description': 'The name of the person', 'type': 'string'}, 'height_in_meters': {'title': 'Height In Meters', 'description': 'The height of the person expressed in meters.', 'type': 'number'}}, 'required': ['name', 'height_in_meters']}}}\`\`\`. Make sure to wrap the answer in \`\`\`json and \`\`\` tagsHuman: Anna is 23 years old and she is 6 feet tall
And here's what it looks like when we invoke it:
chain = prompt | llm | extract_jsonchain.invoke({"query": query})
[{'people': [{'name': 'Anna', 'height_in_meters': 1.8288}]}]Edit this pageWas this page helpful?PreviousHow to route between sub-chainsNextHow to summarize text through parallelizationThe .with_structured_output() methodPydantic classTypedDict or JSON SchemaChoosing between multiple schemasStreamingFew-shot prompting(Advanced) Specifying the method for structuring outputs(Advanced) Raw outputsPrompting and parsing model outputs directlyUsing PydanticOutputParserCustom ParsingCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2024 LangChain, Inc.

In [15]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

Let's start with a simple example containing just a few document pages.

In [17]:
from langchain_chroma import Chroma

vectordb = Chroma.from_documents(langchain_docs, embedding=embeddings) # STORE
vectordb

Definition of a [retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/#:~:text=A%20retriever%20is,Document's%20as%20output.):

> A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [18]:
retriever = vectordb.as_retriever() 
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x11b3f8850>, search_kwargs={})

In [19]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model=MODEL)

In [20]:
# source: https://python.langchain.com/v0.2/docs/tutorials/pdf_qa/#question-answering-with-rag
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [21]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

question_answer_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOllama(model='llama3.2')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

This method `create_stuff_documents_chain` [outputs an LCEL runnable](https://arc.net/l/quote/bnsztwth)

In [22]:
query = "How to return structured output in LangChain?"

In [23]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": query})

results

{'input': 'How to return structured output in LangChain?',
 'context': [Document(metadata={'description': 'This guide assumes familiarity with the following concepts:', 'language': 'en', 'num_chunks': 8, 'source': 'https://python.langchain.com/docs/how_to/functions/', 'title': 'How to run custom functions | 🦜️🔗 LangChain'}, page_content='How to run custom functions | 🦜️🔗 LangChain\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize TextHow-to gu

In [24]:
from IPython.display import Markdown

final_answer = results["answer"]

Markdown(final_answer)

LangChain is a popular library for building data pipelines and workflows in Python. To return structured output in LangChain, you can use the `Result` type from the `langchain chains` module.

Here's an example of how you can use `Result` to return structured output:

```python
from langchain.chains import Result

class MyPipeline:
    def __init__(self):
        self.input_data = None

    def run(self, input_data):
        # Process the input data here...
        processed_data = process_input_data(input_data)

        # Return a Result with the structured output
        return Result(s success=process_data_successful(), output=processed_data)

def process_input_data(input_data):
    # Process the input data here...
    pass

def process_data_successful():
    # Return True if the processing was successful
    return True
```

In this example, `MyPipeline` is a class that takes in some input data and processes it using the `process_input_data` function. The `Result` type is used to wrap the result of the processing operation, with the following fields:

*   `success`: A boolean indicating whether the operation was successful.
*   `output`: The actual output of the operation.

When you return a `Result`, LangChain will automatically unbox it and return the `output` field if the `success` field is `True`.

You can also use the `chain_result` function from the `langchain.chains` module to create a Result with additional metadata:

```python
from langchain.chains import chain_result

class MyPipeline:
    def __init__(self):
        self.input_data = None

    def run(self, input_data):
        # Process the input data here...
        processed_data = process_input_data(input_data)

        # Return a Result with structured output and additional metadata
        return chain_result(
            result=process_data_successful(),
            message="Data processing successful",
            output=processed_data,
            code_status="success"
        )
```

In this case, the `chain_result` function returns a Result with an additional `message`, `code_status` field, in addition to the `output` field.

In [25]:
query_summary = "Write about what is LangChain Expression Language (LCEL)"

 # adding chat history so the model remembers previous questions
output = rag_chain.invoke({"input": query_summary})

Markdown(output["answer"])

LangChain Expression Language (LCEL) is an open-source, human-readable expression language designed specifically for use in LangChain, a blockchain-agnostic data management platform. LCEL allows developers to express complex data operations and transformations in a concise and readable manner.

The main goals of LCEL are:

1. **Simplify data operations**: By providing a standardized way to describe data processing tasks, LCEL makes it easier for developers to write efficient and maintainable code.
2. **Improve readability**: LCEL's syntax is designed to be easy to understand, even for non-technical users, making it ideal for collaboration between developers and data analysts.
3. **Enable data transformation**: LCEL allows developers to define data transformations as a series of steps, enabling the creation of complex workflows that can handle large datasets.

Key features of LCEL include:

1. **Declarative syntax**: LCEL uses a declarative syntax, where the focus is on what needs to be done, rather than how it should be done.
2. **Built-in data types**: LCEL supports various built-in data types, such as arrays, objects, and sets, making it easy to work with different data structures.
3. **Functions and operators**: LCEL provides a range of functions and operators for performing common data operations, such as filtering, mapping, and aggregating data.

Some examples of LCEL expressions include:

* `data | filter(x => x > 5)` (filter an array of numbers greater than 5)
* `data | map(x => x * 2)` (multiply each number in the array by 2)
* `data | reduce((a, b) => a + b, 0)` (calculate the sum of all numbers in the array)

LCEL is designed to be used with LangChain's data management platform, which provides a flexible and scalable way to store, retrieve, and manipulate data. By combining LCEL with LangChain's features, developers can build robust data pipelines that automate complex workflows.

Overall, LangChain Expression Language (LCEL) offers a powerful tool for expressing data operations in a concise, readable, and maintainable way, making it an attractive choice for developers working on blockchain-agnostic data management projects.

The final output is easily verifiable, we can see below that the chunk context for the answer came from pages 0,5,7 and 16 in the source pdf.

In [26]:
for i in range(len(output['context'])):
    print(output['context'][i].metadata)

{'description': 'This guide assumes familiarity with the following concepts:', 'language': 'en', 'num_chunks': 8, 'source': 'https://python.langchain.com/docs/how_to/functions/', 'title': 'How to run custom functions | 🦜️🔗 LangChain'}
{'description': 'Markdown is a lightweight markup language for creating formatted text using a plain-text editor.', 'language': 'en', 'num_chunks': 5, 'source': 'https://python.langchain.com/docs/how_to/document_loader_markdown/', 'title': 'How to load Markdown | 🦜️🔗 LangChain'}
{'description': 'JSON (JavaScript Object Notation) is an open standard file format and data interchange format that uses human-readable text to store and transmit data objects consisting of attribute–value pairs and arrays (or other serializable values).', 'language': 'en', 'num_chunks': 11, 'source': 'https://python.langchain.com/docs/how_to/document_loader_json/', 'title': 'How to load JSON | 🦜️🔗 LangChain'}
{'description': 'This guide assumes familiarity with the following conc

Let's now dig deeper into RAG over the langchain documentation and construct this rag chain ourselves.

In [27]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', '{input}')
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_from_docs = (
    {
        'input': lambda x: x['input'],
        'context': lambda x: format_docs(x['context']), 
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
# passing the input query to the retriever
retrieve_docs = (lambda x: x['input']) | retriever

In [29]:
chain = RunnablePassthrough.assign(context=retrieve_docs).assign(
    answer=rag_chain_from_docs
)
chain

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x11b3f8850>, search_kwargs={})
})
| RunnableAssign(mapper={
    answer: {
              input: RunnableLambda(...),
              context: RunnableLambda(...)
            }
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_varia

In [30]:
query = "In LangChain how do we get structured outputs from a model?" 
chain.invoke({'input': query})

{'input': 'In LangChain how do we get structured outputs from a model?',
 'context': [Document(metadata={'description': 'This guide assumes familiarity with the following concepts:', 'language': 'en', 'num_chunks': 8, 'source': 'https://python.langchain.com/docs/how_to/functions/', 'title': 'How to run custom functions | 🦜️🔗 LangChain'}, page_content='How to run custom functions | 🦜️🔗 LangChain\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize

Adding structured sources:

In [31]:
# source: https://python.langchain.com/v0.3/docs/how_to/qa_sources/
from typing import List

from langchain_core.runnables import RunnablePassthrough
from typing_extensions import Annotated, TypedDict


# Desired schema for response
class AnswerWithSources(TypedDict):
    """An answer to the question, with sources."""

    answer: str
    sources: Annotated[
        List[str],
        ...,
        "List of sources (author + year) used to answer the question",
    ]


# Our rag_chain_from_docs has the following changes:
# - add `.with_structured_output` to the LLM;
# - remove the output parser
rag_chain_from_docs = (
    {
        "input": lambda x: x["input"],
        "context": lambda x: format_docs(x["context"]),
    }
    | prompt
    | llm.with_structured_output(AnswerWithSources)
)

retrieve_docs = (lambda x: x["input"]) | retriever

chain = RunnablePassthrough.assign(context=retrieve_docs).assign(
    answer=rag_chain_from_docs
)

response = chain.invoke({"input": query})
response

{'input': 'In LangChain how do we get structured outputs from a model?',
 'context': [Document(metadata={'description': 'This guide assumes familiarity with the following concepts:', 'language': 'en', 'num_chunks': 8, 'source': 'https://python.langchain.com/docs/how_to/functions/', 'title': 'How to run custom functions | 🦜️🔗 LangChain'}, page_content='How to run custom functions | 🦜️🔗 LangChain\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG ApplicationBuild a PDF ingestion and Question/Answering systemBuild a Retrieval Augmented Generation (RAG) AppVector stores and retrieversBuild a Question/Answering system over SQL dataSummarize

# References

https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb 
Below are notebook from openai cookbook on these topics of search and embeddings:
- https://github.com/openai/openai-cookbook/blob/main/examples/Get_embeddings.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Code_search.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Customizing_embeddings.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Embedding_Wikipedia_articles_for_search.ipynb
- https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
- [In-context learning abilities of ChatGPT models](https://arxiv.org/pdf/2303.18223.pdf)
- [Issue with long context](https://arxiv.org/pdf/2303.18223.pdf)